In [1]:
from IPython.display import display, Math, Latex
import pandas as pd
import numpy as np
#import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import itertools
import copy
from datetime import datetime

In [5]:
MinStocks = 10
MaxStocks = 22
MaxWeight = 0.2
MinWeight = 0.05
Start_date = "2023-01-01"
End_date = "2023-06-30"
TradingFee = 4.95
InvestmentAmount = 750000
ticker_file = "example_tickers2.csv"

In [ ]:
tickers = pd.read_csv(ticker_file, header=None)##Header is none to read in the first element.
tickers.columns=["Tickers"]
exchangeCADUSD = yf.download('CADUSD=x', start=Start_date, end=End_date).tz_localize(None)
tickers[1]

[*********************100%%**********************]  1 of 1 completed


In [13]:
def USDtoCAD(closing):
    #Merges the closing data with the exchange data, then removes all na values (This makes sure each closing price is matched with its dates exchange rate)
    exchanged = pd.concat([closing.to_frame(),exchangeCADUSD["Close"]],axis=1).dropna()

    #Since we are going USD to CAD, we divide by the exchange rate of that day
    converted = exchanged.iloc[:,0]/exchanged.iloc[:,1]
    return converted


In [14]:
## Consumes a series or listof possible stock symboles
def validTickers (tickers):
    ##Creates a new dataframe to hold the valid tickers closing prices
    df = pd.DataFrame()

    #Itterating through the tickers

    for i in tickers:
        #Creating the ticker objects
        ticker = yf.Ticker(i)

        #Try ensures that even if an error occurs, the code will still run (this gets past the delisted stocks)
        try:
            #Running .fast_info['currency'] on a stock will cause an error and stop any non-listed stocks from being ran
            ticker.fast_info['currency']

            #Download the Daily stock history for the given time periods
            history = ticker.history(start=Start_date, end=End_date).tz_localize(None)

            #Creates a seperate dataframe with the values collected by month
            histByMonth = history['Volume'].to_frame().groupby([(history.index.month)])

            #Drops any months with less than 18 trading days
            histByMonth = histByMonth.filter(lambda x: len(x) > 17)

            #Checks for an average monthly volume greater than 150,000
            if histByMonth.sum().mean() > 150000:
                #Checking if the stock is listed in CAD or USD
                if ticker.fast_info['currency'] == 'CAD':

                    #Adding the stock to the valid tickers dataframe
                    df[i] = history['Close']

                elif ticker.fast_info['currency'] == 'USD':

                    #Adding the stocks closing data to the valid tickers dataframe after converting the prices to CAD
                    df[i] = USDtoCAD(history['Close'])
        except:
            #skips and tickers which cause an error
            pass
    return df

In [15]:
stocks_close = validTickers(tickers)
stocks_close

""
